In [ ]:
import pandas as pd
import json
import torch
import random
import torch
import string
from datasets import load_dataset, load_metric
from transformers import T5ForConditionalGeneration,T5Tokenizer
from pathlib import Path
from pandas import json_normalize
from transformers import AutoTokenizer, AutoModel, AdamW
from helpers import tokenize_and_format, flat_accuracy

In [ ]:
dataset = load_dataset("csv", data_files="train_df.csv")

In [ ]:
buff = dataset["train"].train_test_split(test_size=1000)
dataset["validation"] = buff["test"]
dataset["train"] = buff["train"]
buff = dataset["train"].train_test_split(test_size=1000)
dataset["test"] = buff["test"]
dataset["train"] = buff["train"]

In [ ]:
dataset

In [ ]:
dataset["train"] = dataset["train"].shuffle()
dataset["test"] = dataset["test"].shuffle()
dataset["validation"] = dataset["validation"].shuffle()

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the pre-trained BART model and the tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
max_input_length = 512
max_target_length = 64

def preprocess_data(examples):
   inputs = [
      "question: " + text for text in examples["text"]]
   model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

   # Setup the tokenizer for targets
   with tokenizer.as_target_tokenizer():
      labels = tokenizer(examples["label"], max_length=max_target_length, 
                        truncation=True)

   model_inputs["labels"] = labels["input_ids"]


   return model_inputs

In [ ]:

tokenized_datasets = dataset.map(preprocess_data, batched=True)

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(dataset["train"].column_names)

In [ ]:
tokenizer.decode(tokenized_datasets["train"][0]["input_ids"])

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:

batch_size = 8
model_name = "t5-base-legal-qa"
model_dir = f"{model_name}"

args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=200,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    report_to="tensorboard"
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [ ]:
metric = load_metric("rouge")

In [ ]:
import numpy as np
import nltk
nltk.download('punkt')

def compute_metrics(eval_pred):
    breakpoint()
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) 
                      for label in decoded_labels]
    
    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
# Function that returns an untrained model to be trained
def model_init():
    return T5ForConditionalGeneration.from_pretrained("t5-small")

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
# Start TensorBoard before training to monitor it in progress
%load_ext tensorboard
%tensorboard --logdir '{model_dir}'/runs

In [ ]:
trainer.train()